# Credit Risk Resampling Techniques

In [57]:
import warnings
warnings.filterwarnings('ignore')

In [58]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [59]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [60]:
from sklearn.preprocessing import StandardScaler

# Use LabelEnconder on "homeowner" column

# Creating an instance of label encoder
label_encoder = LabelEncoder()

# Fitting the label encoder
label_encoder.fit(df["homeowner"])

# Encode the "homeowner" values as integers
df["homeowner_le"] = label_encoder.transform(df["homeowner"])
df.drop(columns="homeowner", inplace=True)
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status,homeowner_le
0,10700.0,7.672,52800,0.431818,5,1,22800,low_risk,1
1,8400.0,6.692,43600,0.311927,3,0,13600,low_risk,1
2,9000.0,6.963,46100,0.349241,3,0,16100,low_risk,2
3,10700.0,7.664,52700,0.430740,5,1,22700,low_risk,1
4,10800.0,7.698,53000,0.433962,5,1,23000,low_risk,0


# Split the Data into Training and Testing

In [61]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]

In [62]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_le
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.606144
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.667811
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,1.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,2.000000


In [63]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [145]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape

(58152, 8)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [65]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [66]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset

data_scaler.fit(X_train)

StandardScaler()

In [67]:
# Scale the training data

scaled_train = data_scaler.transform(X_train)
scaled_train


array([[ 0.14045627,  0.1230916 ,  0.12859477, ...,  1.04495557,
         0.12859477, -0.90997878],
       [-0.28969527, -0.27393966, -0.27772578, ..., -0.67471435,
        -0.27772578, -0.90997878],
       [-0.48087373, -0.47526712, -0.48088605, ..., -0.67471435,
        -0.48088605, -0.90997878],
       ...,
       [-1.48456066, -1.46728289, -1.47278621, ..., -0.67471435,
        -1.47278621, -0.90997878],
       [-0.28969527, -0.26831599, -0.26577518, ..., -0.67471435,
        -0.26577518, -0.90997878],
       [-1.05440912, -1.05787955, -1.05451506, ..., -0.67471435,
        -1.05451506, -0.90997878]])

# Simple Logistic Regression

In [100]:
from sklearn.linear_model import LogisticRegression

model_sim = LogisticRegression(solver='lbfgs', random_state=1)
model_sim.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [101]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred_sim = model_sim.predict(X_test)
balanced_accuracy_score(y_test, y_pred_sim)

0.9543211898288821

In [102]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_sim)

array([[  571,    54],
       [   93, 18666]], dtype=int64)

In [96]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_sim))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [121]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler


# View the count of target classes with Counter
ros = RandomOverSampler(random_state=1)
X_resampled_over, y_resampled_over = ros.fit_resample(X_train, y_train)
Counter(y_resampled_over)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [122]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model_over = LogisticRegression(solver='lbfgs', random_state=1)
model_over.fit(X_resampled_over, y_resampled_over)

LogisticRegression(random_state=1)

In [123]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred_over = model_over.predict(X_test)
balanced_accuracy_score(y_test, y_pred_over)


0.9948013433551894

In [124]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_over)

array([[  622,     3],
       [  105, 18654]], dtype=int64)

In [125]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred_over))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [126]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled_SMOTE, y_resampled_SMOTE = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)

# View the count of target classes with Counter
from collections import Counter

Counter(y_resampled_SMOTE)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [127]:
# Train the Logistic Regression model using the resampled data
model_over_SMOTE = LogisticRegression(solver='lbfgs', random_state=1)
model_over_SMOTE.fit(X_resampled_SMOTE, y_resampled_SMOTE)

LogisticRegression(random_state=1)

In [128]:
# Calculated the balanced accuracy score
y_pred_SMOTE = model_over_SMOTE.predict(X_test)
balanced_accuracy_score(y_test, y_pred_SMOTE)

0.9948279972279972

In [129]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_SMOTE)

array([[  622,     3],
       [  104, 18655]], dtype=int64)

In [130]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_SMOTE))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [132]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled_under, y_resampled_under = cc.fit_resample(X_train, y_train)



# View the count of target classes with Counter
from collections import Counter

Counter(y_resampled_under)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [133]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model_under = LogisticRegression(solver='lbfgs', random_state=1)
model_under.fit(X_resampled_under, y_resampled_under)

LogisticRegression(random_state=1)

In [134]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred_under = model_under.predict(X_test)
balanced_accuracy_score(y_test, y_pred_under)

0.9828813049736127

In [135]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred_under = model_under.predict(X_test)
confusion_matrix(y_test, y_pred_under)

array([[  607,    18],
       [  102, 18657]], dtype=int64)

In [136]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_under))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.97      0.99      0.91      0.98      0.96       625
   low_risk       1.00      0.99      0.97      1.00      0.98      0.97     18759

avg / total       0.99      0.99      0.97      0.99      0.98      0.97     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [137]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
X_resampled_comb, y_resampled_comb = sm.fit_resample(X_train, y_train)
Counter(y_resampled_comb)

# View the count of target classes with Counter
Counter(y_resampled_comb)

Counter({'high_risk': 55510, 'low_risk': 55930})

In [139]:
# Train the Logistic Regression model using the resampled data

model_comb = LogisticRegression(solver='lbfgs', random_state=1)
model_comb.fit(X_resampled_comb, y_resampled_comb)

LogisticRegression(random_state=1)

In [141]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred_comb = model_comb.predict(X_test)
balanced_accuracy_score(y_test, y_pred_comb)

0.994748035609574

In [146]:
# Display the confusion matrix
y_pred_comb = model_comb.predict(X_test)
confusion_matrix(y_test, y_pred_comb)

array([[  622,     3],
       [  107, 18652]], dtype=int64)

In [147]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred_comb))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

    **SMOTE oversampling had the best score of 0.9948279972279972, which was very close to the Native Undersampling score of 0.9948013433551894. Very close and good models to use!**
    
2. Which model had the best recall score?

    **The average/total recal score for all models were 0.99. However, it was noted that the Undersampling model high_risk was 0.97, where as the other models all were 1.00.**

3. Which model had the best geometric mean score?

    **All models with the exception of the Undersampling (.98) had a geometric mean score of 0.99**